In [62]:
import sys
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [66]:
def log_loss(yt,yp):
    res = np.sum(np.multiply(yt,np.log(yp)) + np.multiply((1-yt), np.log(1-yp)))
    res = -res/len(yp)
    return (res)

In [4]:
df_raw = pd.read_csv('./data/train.csv', index_col = 0)

In [72]:
df_raw.head(10)

,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007
619,2,50,12500,98,9.999999e-01
664,0,13,3250,28,9.999999e-01
441,1,16,4000,35,9.999999e-01
160,2,20,5000,45,9.999999e-01
358,1,24,6000,77,1.000000e-07
335,4,4,1000,4,1.000000e-07
47,2,7,1750,14,9.999999e-01
164,1,12,3000,35,1.000000e-07
736,5,46,11500,98,9.999999e-01
436,0,3,750,4,1.000000e-07


In [5]:
df_raw.loc[df_raw['Made Donation in March 2007'] == 1, 'Made Donation in March 2007'] = 0.9999999
df_raw.loc[df_raw['Made Donation in March 2007'] == 0, 'Made Donation in March 2007'] = 0.0000001

In [6]:
df_raw.dtypes

Months since Last Donation       int64
Number of Donations              int64
Total Volume Donated (c.c.)      int64
Months since First Donation      int64
Made Donation in March 2007    float64
dtype: object

In [7]:
from sklearn.model_selection import train_test_split

In [29]:
train, test = train_test_split(df_raw, test_size = 0.2)
y_true = test['Made Donation in March 2007']

In [9]:
m = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
m.fit(train.drop(['Made Donation in March 2007'], axis=1), train['Made Donation in March 2007'])
y_pred = m.predict(test.drop(['Made Donation in March 2007'], axis=1))
log_loss(y_true, y_pred) 

0.6037572515721014


In [10]:
from catboost import CatBoostRegressor

In [78]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats (containers with only __len__ and __getitem__ defined).
        # weights parameter can be None.
        # Returns list of pairs (der1, der2)
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        exponents = []
        for index in xrange(len(approxes)):
            exponents.append(math.exp(approxes[index]))

        result = []
        for index in xrange(len(targets)):
            p = exponents[index] / (1 + exponents[index])
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))

        return result

In [76]:
q=0.0
for i in range(0,10):
    train, test = train_test_split(df_raw, test_size = 0.2)
    y_true = test['Made Donation in March 2007']
    mc = CatBoostRegressor(iterations=100, max_depth=9, loss_function='MAE', learning_rate=0.1)
    mc.fit(train.drop(['Made Donation in March 2007'], axis=1), train['Made Donation in March 2007'])
    y_pred = mc.predict(test.drop(['Made Donation in March 2007'], axis=1))
    q=q+log_loss(y_true, y_pred)
q=q/10
print(q)

0:	learn: 0.2660468	total: 5.59ms	remaining: 553ms
1:	learn: 0.2481284	total: 21.7ms	remaining: 1.06s
2:	learn: 0.2506990	total: 23.8ms	remaining: 769ms
3:	learn: 0.2499223	total: 35.8ms	remaining: 859ms
4:	learn: 0.2466640	total: 45.9ms	remaining: 872ms
5:	learn: 0.2465570	total: 56ms	remaining: 877ms
6:	learn: 0.2448947	total: 65.6ms	remaining: 871ms
7:	learn: 0.2463092	total: 75.2ms	remaining: 864ms
8:	learn: 0.2438833	total: 84.7ms	remaining: 856ms
9:	learn: 0.2452663	total: 94.6ms	remaining: 852ms
10:	learn: 0.2447018	total: 95.8ms	remaining: 775ms
11:	learn: 0.2431825	total: 105ms	remaining: 771ms
12:	learn: 0.2450797	total: 113ms	remaining: 754ms
13:	learn: 0.2427468	total: 123ms	remaining: 754ms
14:	learn: 0.2429783	total: 127ms	remaining: 720ms
15:	learn: 0.2423464	total: 129ms	remaining: 677ms
16:	learn: 0.2431318	total: 138ms	remaining: 675ms
17:	learn: 0.2418059	total: 140ms	remaining: 636ms
18:	learn: 0.2417199	total: 149ms	remaining: 636ms
19:	learn: 0.2421797	total: 159m

/home/pinak17/miniconda3/envs/snakes/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


0:	learn: 0.2659809	total: 17.9ms	remaining: 1.77s
1:	learn: 0.2476183	total: 42.5ms	remaining: 2.08s
2:	learn: 0.2478143	total: 60.2ms	remaining: 1.94s
3:	learn: 0.2496190	total: 72ms	remaining: 1.73s
4:	learn: 0.2460080	total: 81.6ms	remaining: 1.55s
5:	learn: 0.2484827	total: 86ms	remaining: 1.35s
6:	learn: 0.2445364	total: 88.8ms	remaining: 1.18s
7:	learn: 0.2453445	total: 101ms	remaining: 1.17s
8:	learn: 0.2465781	total: 114ms	remaining: 1.16s
9:	learn: 0.2460319	total: 125ms	remaining: 1.13s
10:	learn: 0.2457819	total: 139ms	remaining: 1.12s
11:	learn: 0.2423134	total: 149ms	remaining: 1.09s
12:	learn: 0.2439007	total: 159ms	remaining: 1.06s
13:	learn: 0.2421304	total: 169ms	remaining: 1.04s
14:	learn: 0.2418487	total: 179ms	remaining: 1.01s
15:	learn: 0.2420766	total: 180ms	remaining: 947ms
16:	learn: 0.2428345	total: 191ms	remaining: 932ms
17:	learn: 0.2423151	total: 193ms	remaining: 881ms
18:	learn: 0.2403671	total: 198ms	remaining: 844ms
19:	learn: 0.2412365	total: 202ms	rema

68:	learn: 0.2238155	total: 613ms	remaining: 276ms
69:	learn: 0.2235613	total: 616ms	remaining: 264ms
70:	learn: 0.2226561	total: 620ms	remaining: 253ms
71:	learn: 0.2225458	total: 637ms	remaining: 248ms
72:	learn: 0.2230876	total: 644ms	remaining: 238ms
73:	learn: 0.2210140	total: 657ms	remaining: 231ms
74:	learn: 0.2213353	total: 667ms	remaining: 222ms
75:	learn: 0.2201217	total: 677ms	remaining: 214ms
76:	learn: 0.2210605	total: 682ms	remaining: 204ms
77:	learn: 0.2200147	total: 691ms	remaining: 195ms
78:	learn: 0.2199771	total: 702ms	remaining: 186ms
79:	learn: 0.2180964	total: 711ms	remaining: 178ms
80:	learn: 0.2198948	total: 722ms	remaining: 169ms
81:	learn: 0.2182366	total: 734ms	remaining: 161ms
82:	learn: 0.2180425	total: 742ms	remaining: 152ms
83:	learn: 0.2168221	total: 753ms	remaining: 143ms
84:	learn: 0.2163410	total: 763ms	remaining: 135ms
85:	learn: 0.2174947	total: 776ms	remaining: 126ms
86:	learn: 0.2165601	total: 790ms	remaining: 118ms
87:	learn: 0.2136703	total: 801

47:	learn: 0.2182021	total: 405ms	remaining: 439ms
48:	learn: 0.2187520	total: 411ms	remaining: 428ms
49:	learn: 0.2184320	total: 413ms	remaining: 413ms
50:	learn: 0.2196730	total: 415ms	remaining: 399ms
51:	learn: 0.2183307	total: 429ms	remaining: 396ms
52:	learn: 0.2184672	total: 432ms	remaining: 383ms
53:	learn: 0.2171343	total: 436ms	remaining: 372ms
54:	learn: 0.2180843	total: 450ms	remaining: 368ms
55:	learn: 0.2166685	total: 459ms	remaining: 361ms
56:	learn: 0.2172381	total: 469ms	remaining: 354ms
57:	learn: 0.2159497	total: 479ms	remaining: 347ms
58:	learn: 0.2163731	total: 480ms	remaining: 334ms
59:	learn: 0.2166457	total: 485ms	remaining: 323ms
60:	learn: 0.2146156	total: 495ms	remaining: 317ms
61:	learn: 0.2140822	total: 503ms	remaining: 308ms
62:	learn: 0.2139186	total: 513ms	remaining: 301ms
63:	learn: 0.2156378	total: 522ms	remaining: 294ms
64:	learn: 0.2142238	total: 532ms	remaining: 286ms
65:	learn: 0.2136387	total: 534ms	remaining: 275ms
66:	learn: 0.2144201	total: 536

20:	learn: 0.2430703	total: 217ms	remaining: 818ms
21:	learn: 0.2432647	total: 229ms	remaining: 813ms
22:	learn: 0.2421170	total: 232ms	remaining: 776ms
23:	learn: 0.2425248	total: 244ms	remaining: 774ms
24:	learn: 0.2424299	total: 254ms	remaining: 763ms
25:	learn: 0.2414333	total: 266ms	remaining: 757ms
26:	learn: 0.2413601	total: 276ms	remaining: 745ms
27:	learn: 0.2414718	total: 280ms	remaining: 720ms
28:	learn: 0.2407768	total: 283ms	remaining: 692ms
29:	learn: 0.2403831	total: 284ms	remaining: 664ms
30:	learn: 0.2401792	total: 295ms	remaining: 656ms
31:	learn: 0.2376120	total: 305ms	remaining: 648ms
32:	learn: 0.2372403	total: 314ms	remaining: 638ms
33:	learn: 0.2360458	total: 324ms	remaining: 629ms
34:	learn: 0.2385581	total: 333ms	remaining: 619ms
35:	learn: 0.2373044	total: 335ms	remaining: 595ms
36:	learn: 0.2370906	total: 339ms	remaining: 577ms
37:	learn: 0.2368474	total: 349ms	remaining: 569ms
38:	learn: 0.2368136	total: 357ms	remaining: 558ms
39:	learn: 0.2364073	total: 367

94:	learn: 0.2081959	total: 810ms	remaining: 42.6ms
95:	learn: 0.2069026	total: 822ms	remaining: 34.3ms
96:	learn: 0.2065709	total: 834ms	remaining: 25.8ms
97:	learn: 0.2073837	total: 839ms	remaining: 17.1ms
98:	learn: 0.2057778	total: 849ms	remaining: 8.57ms
99:	learn: 0.2064921	total: 861ms	remaining: 0us
0:	learn: 0.2700838	total: 6.17ms	remaining: 610ms
1:	learn: 0.2542812	total: 21.9ms	remaining: 1.07s
2:	learn: 0.2547859	total: 33ms	remaining: 1.07s
3:	learn: 0.2543251	total: 55.9ms	remaining: 1.34s
4:	learn: 0.2519715	total: 82.5ms	remaining: 1.57s
5:	learn: 0.2494202	total: 111ms	remaining: 1.74s
6:	learn: 0.2509105	total: 130ms	remaining: 1.73s
7:	learn: 0.2498540	total: 136ms	remaining: 1.57s
8:	learn: 0.2483079	total: 149ms	remaining: 1.51s
9:	learn: 0.2499328	total: 152ms	remaining: 1.37s
10:	learn: 0.2470144	total: 173ms	remaining: 1.4s
11:	learn: 0.2477829	total: 177ms	remaining: 1.29s
12:	learn: 0.2492317	total: 182ms	remaining: 1.22s
13:	learn: 0.2471711	total: 199ms	re

60:	learn: 0.2330871	total: 1.04s	remaining: 667ms
61:	learn: 0.2321645	total: 1.05s	remaining: 645ms
62:	learn: 0.2332450	total: 1.07s	remaining: 630ms
63:	learn: 0.2317811	total: 1.09s	remaining: 613ms
64:	learn: 0.2314752	total: 1.1s	remaining: 594ms
65:	learn: 0.2310436	total: 1.12s	remaining: 578ms
66:	learn: 0.2302568	total: 1.13s	remaining: 558ms
67:	learn: 0.2291800	total: 1.14s	remaining: 536ms
68:	learn: 0.2306814	total: 1.15s	remaining: 518ms
69:	learn: 0.2286627	total: 1.15s	remaining: 495ms
70:	learn: 0.2273605	total: 1.16s	remaining: 475ms
71:	learn: 0.2272442	total: 1.17s	remaining: 456ms
72:	learn: 0.2248405	total: 1.18s	remaining: 437ms
73:	learn: 0.2246453	total: 1.19s	remaining: 419ms
74:	learn: 0.2252739	total: 1.2s	remaining: 400ms
75:	learn: 0.2236488	total: 1.21s	remaining: 382ms
76:	learn: 0.2236104	total: 1.21s	remaining: 362ms
77:	learn: 0.2237874	total: 1.22s	remaining: 343ms
78:	learn: 0.2238522	total: 1.23s	remaining: 326ms
79:	learn: 0.2232006	total: 1.23s

In [49]:
df_test = pd.read_csv('data/test.csv', index_col = 0)

In [71]:
mct = CatBoostRegressor(iterations=100, max_depth=9, loss_function='MAE', learning_rate=0.1)
mct.fit(df_raw.drop(['Made Donation in March 2007'], axis=1), df_raw['Made Donation in March 2007'])
test_pred = mct.predict(df_test)

0:	learn: 0.2649305	total: 11.5ms	remaining: 1.14s
1:	learn: 0.2484175	total: 15.8ms	remaining: 773ms
2:	learn: 0.2534187	total: 32.6ms	remaining: 1.05s
3:	learn: 0.2488817	total: 42.5ms	remaining: 1.02s
4:	learn: 0.2477781	total: 50.7ms	remaining: 964ms
5:	learn: 0.2548624	total: 54.3ms	remaining: 850ms
6:	learn: 0.2434278	total: 62.6ms	remaining: 831ms
7:	learn: 0.2471840	total: 70.4ms	remaining: 809ms
8:	learn: 0.2469820	total: 78.1ms	remaining: 790ms
9:	learn: 0.2452155	total: 86ms	remaining: 774ms
10:	learn: 0.2475467	total: 89.6ms	remaining: 725ms
11:	learn: 0.2446145	total: 91.9ms	remaining: 674ms
12:	learn: 0.2436332	total: 99.6ms	remaining: 667ms
13:	learn: 0.2448064	total: 102ms	remaining: 626ms
14:	learn: 0.2432401	total: 110ms	remaining: 622ms
15:	learn: 0.2426778	total: 116ms	remaining: 608ms
16:	learn: 0.2442367	total: 124ms	remaining: 604ms
17:	learn: 0.2395786	total: 131ms	remaining: 599ms
18:	learn: 0.2427539	total: 139ms	remaining: 594ms
19:	learn: 0.2407363	total: 14

In [72]:
test_pred = mct.predict(df_test)

In [73]:
df_test['Made Donation in March 2007'] = test_pred

In [74]:
df_test.head(10)

,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007
659,2,12,3000,52,0.234800
276,21,7,1750,38,0.017983
263,4,1,250,4,-0.017285
303,11,11,2750,38,0.061245
83,4,12,3000,34,0.308557
500,3,21,5250,42,1.046725
530,4,2,500,4,0.025360
244,14,1,250,14,-0.003581
249,23,2,500,87,-0.012473
728,14,4,1000,64,-0.031826


In [75]:
df_test.to_csv('resultscaler.csv', columns=['Made Donation in March 2007'])